## Imports

In [1]:
# imports
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import wandb
#from prettytable import PrettyTable
import random
import os
from datetime import datetime
from torch.optim import lr_scheduler

# local imports
from models_parameters import losses
from models_parameters import models
from utils import helper_functions_srcnn as helper_functions
from utils.dataloader import Dataset as dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

C:\Users\accou\anaconda3\envs\geo_env3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


# GENERAL SETTINGS

In [2]:
# Running where? "colab" or "ubs"
location = "local"

# chose which tile to train, test on eg "T30UXU"(9k) or "T30UUU"(3k)
sen2_tile_train = "all"
sen2_tile_val   = "all"
sen2_tile_test  = "all"

# chose dataloader transform - 'standardize', 'histogram_matching', 'moment_matching', 'interpolate' or 'spot6'
transform = "interpolate"

# chose where and how often to output - 'show' or output path 
# /share/projects/erasmus/sesure/working_dir/images/
output_location = "C:\\Users\\accou\\Documents\\thesis\\images\\images_SRCNN_sat_channel\\"

# selesct wether to load from checkpoint - 'None' for no checkpoint or checkpoint path
load_checkpoint = "None"

# Save checkpoints, 'None' to not save, otherwise path
save_checkpoint = True

# set WandB Project Name
model_name = "SRCNN_local_sat_channel_"

In [3]:
# create run_name
run_name = model_name+"_"+transform+"_"+sen2_tile_train+"__"+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))
print(run_name)

SRCNN_local_sat_channel__interpolate_all__15-05-2022_03-05-10


# MODEL SETTINGS

In [4]:
# Load Model, wither straight definitions or loaded models
#model = models.RCAN.create_model(args)
model = models.SRCNN_channel()

# load pretrained model weights
if load_checkpoint!="None":
  model.load_state_dict(torch.load(load_checkpoint))

# Load Loss Function
loss_func = losses.loss_mse

# enable benchmark for faster execution (only if inputs dont change)
torch.backends.cudnn.benchmark = True 

In [5]:
# TRANING SETTINGS
batch_size = 32
lr = 0.000001
epochs = 500
plot_frequency = 50 # in batches

In [6]:
model.children

<bound method Module.children of SRCNN_channel(
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU(inplace=True)
)>

## Create Dataset Object

In [7]:
if location=="colab":
  working_directory = "/content/drive/MyDrive/thesis/"
  folder_path = "/content/drive/MyDrive/thesis/data/"
  dataset_file = "/content/drive/MyDrive/thesis/data/images_subfolders3.pkl"
if location=="ubs":
  working_directory = "/share/projects/erasmus/sesure/working_dir/"
  folder_path = "/share/projects/erasmus/sesure/working_dir/"
  dataset_file = "/share/projects/erasmus/sesure/working_dir/df_saved_images.pkl"
if location=="local":
  working_directory = "C:\\Users\\accou\\Documents\\thesis\\"
  folder_path = "C:\\Users\\accou\\Documents\\thesis\\data\\"
  dataset_file = "C:\\Users\\accou\\Documents\\thesis\\images_subfolders3.pkl"

# get dataset object
dataset_train = dataset(folder_path,dataset_file,transform,sen2_amount=1, sen2_tile = sen2_tile_train, location=location)
dataset_val   = dataset(folder_path,dataset_file,transform,sen2_amount=1, sen2_tile = sen2_tile_val,   location=location)
#dataset_test  = dataset(folder_path,dataset_file,transform,sen2_amount=1, sen2_tile = sen2_tile_test,  location=location)

## Train function

In [8]:
# implementation of model trainer function
def train_model(run_name,model,batch_size=1,lr=0.001,epochs=10,wandb_name="test",plot_frequency=10):
    
    logging=True          # log to WandB
    logging_val=True      # log validation loss in paralel to training loss
    plotting=True         # set if images should be plotted
    plot_frequency = plot_frequency    # set after how many batches images shoud be plotted

    if logging==True:
        wandb.init(name=run_name,project=wandb_name, entity="simon-donike")
        wandb.config = {
          "learning_rate": lr,
          "epochs": epochs,
          "batch_size": batch_size
        }
    
    # define loaders
    
    loader_train = DataLoader(dataset_train,batch_size=batch_size, shuffle=True, num_workers=1,prefetch_factor=4,pin_memory=True)
    loader_val  = DataLoader(dataset_val,batch_size=1, shuffle=True, num_workers=1)
    print("dataloader instanciated!")
    print("Len. Train: ",len(loader_train),"(Batch Sz. "+str(batch_size),") Len. Val: ",len(dataset_val),"(Batch Sz. "+str(1)+")")
    """
    from utils.dataloader_urban_interpolate import Dataset
    dataset_train = Dataset()
    loader_train = DataLoader(dataset_train,batch_size=batch_size,shuffle="true")
    loader_val = DataLoader(dataset_train,batch_size=batch_size,shuffle="true")
    print("len train loader: ",len(loader_train))
    """
    train_loss = []  # where we keep track of the training loss
    train_accuracy = []  # where we keep track of the training accuracy of the model
    val_loss = []  # where we keep track of the validation loss
    val_accuracy = []  # where we keep track of the validation accuracy of the model
    epochs = epochs  # number of epochs

    # initialize model
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr,momentum=0.9)
    scheduler = lr_scheduler.StepLR(optimizer, 100, 0.1,verbose=True)
    #optimizer = optim.Adam(model.parameters())
    
    for e in range(epochs):
        model.train()
        #train_correct = 0
        batch_count=0
        for (x_train_batch, y_train_batch) in loader_train:
            batch_count = batch_count+1
            
            # send data to device
            x_train_batch = x_train_batch.to(device)
            y_train_batch = y_train_batch.to(device) # to float

            # forward pass
            # pred channel-wise
            pred_0 = model(x_train_batch[0][0].unsqueeze(0))
            pred_1 = model(x_train_batch[0][1].unsqueeze(0))
            pred_2 = model(x_train_batch[0][2].unsqueeze(0))
            pred = torch.stack((pred_0,pred_1,pred_2),dim=1)
            for i in x_train_batch[1:]:
                pred_0 = model(i[0].unsqueeze(0))
                pred_1 = model(i[0].unsqueeze(0))
                pred_2 = model(i[0].unsqueeze(0))
                pred_c = torch.stack((pred_0,pred_1,pred_2),dim=1)
                pred = torch.cat((pred,pred_c))
            y_hat = pred
            
            # compute the loss
            loss = loss_func(y_hat, y_train_batch)  

            loss.backward()  # obtain the gradients with respect to the loss
            optimizer.step()  # perform one step of gradient descent
            optimizer.zero_grad()  # reset the gradients to 0

            """ START LOGGING & PLOTTING """
            # Log train loss if not logging train+val loss
            if logging==True:
                wandb.log({'loss_train_step': loss.item() / len(x_train_batch)})
            # log val loss together with train and val loss
            if logging_val and logging:
              model.eval()
              x_val_batch, y_val_batch = next(iter(loader_val))
              x_val_batch = x_val_batch.to(torch.float).to(device)
              y_val_batch = y_val_batch.to(torch.float).to(device)
              #y_hat_val = model(x_val_batch)
              pred_0 = model(x_val_batch[0][0].unsqueeze(0))
              pred_1 = model(x_val_batch[0][1].unsqueeze(0))
              pred_2 = model(x_val_batch[0][2].unsqueeze(0))
              pred = torch.stack((pred_0,pred_1,pred_2),dim=1)
              for i in x_train_batch[1:]:
                pred_0 = model(i[0].unsqueeze(0))
                pred_1 = model(i[0].unsqueeze(0))
                pred_2 = model(i[0].unsqueeze(0))
                pred_c = torch.stack((pred_0,pred_1,pred_2),dim=1)
                pred = torch.cat((pred,pred_c))
              y_hat_val = pred
              loss_val = loss_func(y_hat_val, y_val_batch) 
              wandb.log({'loss_val_step': loss_val.item() / len(y_val_batch)})
              del x_val_batch
              del y_val_batch
              model.train()

            # log and plot after each batch/after plotting frequency
            if plotting==True:
                if batch_count%plot_frequency==0:
                    print('Epoch', e, ' Batch',batch_count,' finished.  No. of trained Images: '+str(batch_count*batch_size))
                    psnr_int = round(losses.loss_psnr(y_train_batch,helper_functions.interpolate_tensor(y_hat)).item(),2)
                    psnr_pred = round(losses.loss_psnr(y_train_batch,y_hat).item(),2)
                    
                    helper_functions.plot_tensors_window(y_train_batch,x_train_batch,y_hat,psnr_int,psnr_pred,fig_path=output_location)
                    print("Tensor Mean:",round(y_hat.mean().item(),6),"Max: ",round(y_hat.max().item(),6),"Min: ",round(y_hat.min().item(),6))
                    



        """ END OF EPOCH, logging and plotting """
        print ('Epoch', e+1, ' training finished.')
        scheduler.step()
        
        if save_checkpoint:
          torch.save(model.state_dict(), working_directory+"//checkpoints//"+run_name+"_epoch"+str(e)+".pkl")
        """
        PERFORM VALIDATION STEP after each epoch
        """
        model.eval()
        # perform inference 10 times on random images
        psnr_inter_list = []
        psnr_pred_list = []
        loss_val_list = []
        for i in range(5):
          # Load val Data and to device
          x_val_batch, y_val_batch = next(iter(loader_val))
          x_val_batch = x_val_batch.to(torch.float).to(device)
          y_val_batch = y_val_batch.to(torch.float).to(device)
        
          # perform inference step
          with torch.no_grad():
            #pred = model(x_val_batch)
            pred_0 = model(x_val_batch[0][0].unsqueeze(0))
            pred_1 = model(x_val_batch[0][1].unsqueeze(0))
            pred_2 = model(x_val_batch[0][2].unsqueeze(0))
            pred = torch.stack((pred_0,pred_1,pred_2),dim=1)
            for i in x_train_batch[1:]:
                pred_0 = model(i[0].unsqueeze(0))
                pred_1 = model(i[0].unsqueeze(0))
                pred_2 = model(i[0].unsqueeze(0))
                pred_c = torch.stack((pred_0,pred_1,pred_2),dim=1)
                pred = torch.cat((pred,pred_c))
          # calculate metrics
          val_loss = loss_func(pred, y_val_batch).item()
          psnr_int = round(losses.loss_psnr(y_val_batch,helper_functions.interpolate_tensor(x_val_batch)).item(),2)
          psnr_pred = round(losses.loss_psnr(y_val_batch,pred).item(),2)
          psnr_inter_list.append(psnr_int)
          psnr_pred_list.append(psnr_pred)
          loss_val_list.append(val_loss)
        
        # TODO: implement SSIM
        print("VALIDATION: Visualizing 1 of 10 inference steps. Values are averages. Time: ",datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
        psnr_int_avg = round(sum(psnr_inter_list) / len(psnr_inter_list),2)
        psnr_pred_avg = round(sum(psnr_pred_list) / len(psnr_pred_list),2)
        loss_val_avg = round(sum(loss_val_list) / len(loss_val_list),2)
        helper_functions.plot_tensors_window(y_val_batch,x_val_batch,pred,psnr_int_avg,psnr_pred_avg,fig_path=output_location)
        # return model to train state
        model.train()

        # log validation accuracy after each epoch
        if logging==True:
            # log val loss
            wandb.log({'val_loss_epoch': loss_val_avg})
            # log val PSNR
            wandb.log({'val_PSNR_epoch': psnr_pred_avg,
                       'epoch':e,
                       'lr':scheduler.get_lr()[0]})

    # Steps to perform after training is finished
    print("Training finished!")
    if logging==True:
        wandb.finish()

# Perform Training

In [ ]:
train_model(run_name,model=model,batch_size=batch_size,lr=lr,epochs=epochs,wandb_name=model_name,plot_frequency=plot_frequency)

wandb: Currently logged in as: simon-donike (use `wandb login --relogin` to force relogin)


dataloader instanciated!
Len. Train:  1419 (Batch Sz. 32 ) Len. Val:  45380 (Batch Sz. 1)
Adjusting learning rate of group 0 to 1.0000e-06.


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 0  Batch 50  finished.  No. of trained Images: 1600
Tensor Mean: 0.024906 Max:  0.048396 Min:  0.006204


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 3  Batch 1100  finished.  No. of trained Images: 35200
Tensor Mean: 0.029997 Max:  0.073946 Min:  -0.117041


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 3  Batch 1150  finished.  No. of trained Images: 36800
Tensor Mean: 0.030023 Max:  0.049501 Min:  0.00604


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000202BC173168>
Traceback (most recent call last):
  File "C:\Users\accou\anaconda3\envs\geo_env3\lib\site-packages\torch\utils\data\dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "C:\Users\accou\anaconda3\envs\geo_env3\lib\site-packages\torch\utils\data\dataloader.py", line 1322, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\accou\anaconda3\envs\geo_env3\lib\multiprocessing\process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "C:\Users\accou\anaconda3\envs\geo_env3\lib\multiprocessing\popen_spawn_win32.py", line 104, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


# Playground

In [ ]:
    loader_train = DataLoader(dataset_train,batch_size=batch_size, shuffle=True, num_workers=1,prefetch_factor=4,pin_memory=True)
    loader_val  = DataLoader(dataset_val,batch_size=1, shuffle=True, num_workers=1)
    print("dataloader instanciated!")
    print("Len. Train: ",len(loader_train),"(Batch Sz. "+str(batch_size),") Len. Val: ",len(dataset_val),"(Batch Sz. "+str(1)+")")

In [ ]:
len(loader_train)

In [ ]:
a,b = next(iter(loader_train))

In [ ]:
print(working_directory)

In [ ]:

# pred channel-wise
pred_0 = model(a[0][0].unsqueeze(0))
pred_1 = model(a[0][1].unsqueeze(0))
pred_2 = model(a[0][2].unsqueeze(0))
pred = torch.stack((pred_0,pred_1,pred_2),dim=1)
for i in a[1:]:
    pred_0 = model(i[0].unsqueeze(0))
    pred_1 = model(i[0].unsqueeze(0))
    pred_2 = model(i[0].unsqueeze(0))
    pred_c = torch.stack((pred_0,pred_1,pred_2),dim=1)
    pred = torch.cat((pred,pred_c))

In [ ]:
model(i[0].unsqueeze(0))

In [ ]:
pred.size()

In [ ]:
pred_2 = model(a[0][0].unsqueeze(0))